In [ ]:
import time
import ipywidgets as widgets
import numpy as np
import magpylib as magpy
import plotly.graph_objects as go
from magpylib.source.magnet import Box, Cylinder, Sphere
from magpylib.source.current import Line, Circular
from magpylib.source.moment import Dipole
from numpy import array, float64, isnan
from magpylib._lib.classes.plotlysources import makeBox

In [ ]:
def _test_array_len(a, positive=False, length=3):
    try:
        Ex = ValueError()
        if isinstance(a,(float,int,str)):
            Ex.strerror = f"value needs to be a tuple, a list or a numpy array of length {length}"
            raise Ex
        elif isinstance(a,(np.ndarray, np.generic, tuple, list)):
            if len(a)!=length:
                Ex.strerror = f"value needs to be of length {length}"
                raise Ex
            elif sum(n < 0 for n in a) != 0 and positive == True:
                Ex.strerror = "dimensions need to be positive"
                raise Ex 
        else:
            Ex.strerror = f'{type(a)} is not a valid value type, must be one of \n - tuple, list or numpy array of length {length}'
    except ValueError as e:
        print(e.strerror)

class SourcePropertiesArray(np.ndarray):

    def __new__(cls, input_array, positive=False, property_type=None, dtype=float64, copy=False):   
        _test_array_len(input_array, positive=positive)
        obj = np.asarray(input_array, dtype=dtype).view(cls)
        obj.copy=copy
        obj.x = input_array[0]
        obj.y = input_array[1]
        obj.z = input_array[2]
        obj.prop = property_type
        return obj
    
    @property
    def x(self):
        return self[0]

    @x.setter
    def x(self, value):
        self[0] = float(value)
        
    @property
    def y(self):
        return self[1]

    @y.setter
    def y(self, value):
        self[1] = float(value)
        
    @property
    def z(self):
        return self[2]

    @z.setter
    def z(self, value):
        self[2]= float(value)
    
    def __array_finalize__(self, obj):
        if obj is None: return
        self.prop  = getattr(obj, 'prop', None)
        self._x = getattr(obj, 'x', None)
        self._y = getattr(obj, 'y', None)
        self._z = getattr(obj, 'z', None)
        
    

class SourceOrientation:
    def __init__(self, angle=0, axis=(0,0,1)):
        self._angle = float(angle)
        self._axis = SourcePropertiesArray(axis, property_type='axis')
    @property
    def axis(self):
        return self._axis
    
    @axis.setter
    def axis(self, value):
        self._axis = SourcePropertiesArray(value, property_type='axis')
        
    @property
    def angle(self):
        return self._angle
    
    @angle.setter
    def angle(self, value):
        self._angle = float(value)
    
    def __repr__(self):
        return '''angle : {} \naxis  : ({}, {}, {})'''.format(self.angle, *self.axis)

In [ ]:
class RCS:
    """
    FUNDAMENTAL CLASS - RCS (RELATIVE COORDINATE SYSTEM)

    initiates position, orientation
    - adds moveBY, rotateBy
    """

    def __init__(self, position, angle, axis, fig=None):
        # fundamental (unit)-orientation/rotation is [0,0,0,1]
        self._fig = fig
        if self._fig!=None:
            self.fig = fig
        
        self.position = position
        self._orientation = SourceOrientation(angle, axis)
        
    
    def update_trace(self):
        self.trace.update(makeBox(pos=self.position, angle=self.angle, axis=self.axis))
        
    @property
    def fig(self):
        return self._fig

    @fig.setter
    def fig(self, newfig):
        if isinstance(newfig, go.FigureWidget) and newfig is not self._fig:
            self._fig = newfig
            self._fig.add_trace(makeBox(pos=self.position, angle=self.angle, axis=self.axis))
            self.trace = self._fig.data[-1]
        elif newfig == None:
            self._fig = None
        elif newfig is not self._fig:
            raise ValueError(f'{newfig} is not a plotly FigureWidget Class')
        
    @property
    def position(self):
        return self._position

    @position.setter
    def position(self, newPos):
        self.setPosition(newPos)
    
    @property
    def axis(self):
        return self.orientation.axis
    
    @axis.setter
    def axis(self, newaxis):
        self.setOrientation(self._orientation.angle, newaxis)
        
    @property
    def angle(self):
        return self.orientation.angle
    
    @angle.setter
    def angle(self, newangle):
        self.setOrientation(newangle, self._orientation.axis)
        
    @property
    def orientation(self):
        return self._orientation

    @orientation.setter
    def orientation(self, value):
        if isinstance(value, SourceOrientation):
            self.setOrientation(value.angle, value.axis)
        else:
            self.setOrientation(value[0], value[1])
    
    
    def setPosition(self, newPos):
        """
        This method moves the source to the position given by the argument 
        vector `newPos`. Vector input format can be either list, tuple or array
        of any data type (float, int)

        Parameters
        ----------
        newPos : vec3 [mm]
            Set new position of the source.

        Returns
        -------
        None

        Example
        -------
        >>> from magpylib import source
        >>> pm = source.magnet.Sphere(mag=[0,0,1000],dim=1)
        >>> print(pm.position)
            [0. 0. 0.]
        >>> pm.setPosition([5,5,5])
        >>> print(pm.position)
            [5. 5. 5.]
        """
        self._position = SourcePropertiesArray(newPos, dtype=float64, property_type='position')
        
        if self.fig!=None: self.update_trace()
    
    def move(self, displacement):
        """
        This method moves the source by the argument vector `displacement`. 
        Vector input format can be either list, tuple or array of any data
        type (float, int).

        Parameters
        ----------
        displacement : vec3 [mm]
            Set displacement vector

        Returns
        -------
        None

        Example
        -------
        >>> from magpylib import source
        >>> pm = source.magnet.Sphere(mag=[0,0,1000],dim=1,pos=[1,2,3])
        >>> print(pm.position)
            [1. 2. 3.]
        >>> pm.move([3,2,1])
        >>> print(pm.position)
            [4. 4. 4.]
        """
        mV = array(displacement, dtype=float64, copy=False)
        if any(isnan(mV)) or len(mV) != 3:
            sys.exit('Bad move vector input')
        self.position = self.position + mV
        
        if self.fig!=None: self.update_trace()
        

    def setOrientation(self, angle, axis):
        """
        This method sets a new source orientation given by `angle` and `axis`.
        Scalar input is either integer or float. Vector input format can be
        either list, tuple or array of any data type (float, int).

        Parameters
        ----------
        angle  : scalar [deg]
            Set new angle of source orientation.

        axis : vec3 []
            Set new axis of source orientation.

        Returns
        -------
        None            

        Example
        -------
        >>> from magpylib import source
        >>> pm = source.magnet.Sphere(mag=[0,0,1000],dim=1)
        >>> print([pm.angle,pm.axis])
            [0.0, array([0., 0., 1.])]
        >>> pm.setOrientation(45,[0,1,0])
        >>> print([pm.angle,pm.axis])
            [45.0, array([0., 1., 0.])]
        """
        
        self._orientation = SourceOrientation(angle, axis)
        self._axis, self._angle = self._orientation.axis, self._orientation.angle
        
        if self.fig!=None: self.update_trace()

    def rotate(self, angle, axis, anchor='self.position'):
        """
        This method rotates the source about `axis` by `angle`. The axis passes
        through the center of rotation anchor. Scalar input is either integer or
        float. Vector input format can be either list, tuple or array of any
        data type (float, int).

        Parameters
        ----------
        angle  : scalar [deg]
            Set angle of rotation in units of [deg]
        axis : vec3 []
            Set axis of rotation
        anchor : vec3 [mm]
            Specify the Center of rotation which defines the position of the
            axis of rotation. If not specified the source will rotate about its
            own center.

        Returns
        -------
        None

        Example
        -------
        >>> from magpylib import source
        >>> pm = source.magnet.Sphere(mag=[0,0,1000], dim=1)
        >>> print(pm.position, pm.angle, pm.axis)
          [0. 0. 0.] 0.0 [0. 0. 1.]
        >>> pm.rotate(90, [0,1,0], anchor=[1,0,0])
        >>> print(pm.position, pm.angle, pm.axis)
          [1., 0., 1.] 90.0 [0., 1., 0.]
        """
        # secure type
        ax = array(axis, dtype=float64, copy=False)

        try:
            ang = float(angle)
        except ValueError:
            sys.exit('Bad angle input')

        if str(anchor) == 'self.position':
            anchor = self.position
        else:
            anchor = array(anchor, dtype=float64, copy=False)

        # check input
        if any(isnan(ax)) or len(ax) != 3:
            sys.exit('Bad axis input')
        if fastSum3D(ax**2) == 0:
            sys.exit('Bad axis input')
        if any(isnan(anchor)) or len(anchor) != 3:
            sys.exit('Bad anchor input')

        # determine Rotation Quaternion Q from self.axis-angle
        Q = getRotQuat(self.angle, self.axis)

        # determine rotation Quaternion P from rot input
        P = getRotQuat(ang, ax)

        # determine new orientation quaternion which follows from P.Q v (P.Q)*
        R = Qmult(P, Q)

        # reconstruct new axis-angle from new orientation quaternion
        ang3 = arccosSTABLE(R[0])*180/pi*2

        ax3 = R[1:]  # konstanter mult faktor ist wurscht für ax3
        self.angle = ang3
        if ang3 == 0:  # avoid returning a [0,0,0] axis
            self.axis = array([0, 0, 1])
        else:
            Lax3 = fastNorm3D(ax3)
            self.axis = array(ax3)/Lax3

        # set new position using P.v.P*
        posOld = self.position-anchor
        Vold = [0] + [p for p in posOld]
        Vnew = Qmult(P, Qmult(Vold, Qconj(P)))
        self.position = array(Vnew[1:])+anchor
        
        if self.fig!=None: self.update_trace()


In [ ]:
fig1 = go.FigureWidget(layout_width=500)
fig2 = go.FigureWidget(layout_width=500)
s = RCS((0,0,0), 0, (0,0,1))
widgets.HBox([fig1,fig2])

In [ ]:
s.fig = fig1

In [ ]:
for i in range(10):
    time.sleep(1/10)
    s.angle += 1

In [ ]:
s.fig = fig2
for i in range(10):
    time.sleep(1/10)
    s.angle += 1

In [ ]:
s.position.x = 3

In [ ]:
a

In [ ]:
s.position = 1,2,30

In [ ]:
s.orientation = (1,(6,7,8))

In [ ]:
s.orientation.axis = (0,9,9)

In [ ]:
a = s.orientation

In [ ]:
print(f'position = {s.position} \nangle = {s.angle} \naxis = {s.axis}')

In [ ]:
class BoxMagnet:
    def __init__(self, mag=(0,0,1), dim=(1,1,1), pos=(0,0,0), angle=0, axis=(0,0,1)):
        self._magnetization = SourcePropertiesArray(mag, property_type='magnetization')
        self._position = SourcePropertiesArray(pos, property_type='position')
        self._dimension = SourcePropertiesArray(dim, property_type='dimension')
        self._axis = SourcePropertiesArray(axis, property_type='axis')
        self._angle = angle
        self._orientation = SourceOrientation(angle, axis)
        self.name = 'Magnet'
        self.magnet = magpy.source.magnet.Box(mag, dim, pos, angle,  axis)
        
    @property
    def magnetization(self):
        return self._magnetization
    
    @magnetization.setter
    def magnetization(self, value):
        self._magnetization = SourcePropertiesArray(value, property_type='magnetization')
        
        
    @property
    def position(self):
        return self._position

    @position.setter
    def position(self, value):
        self._position = SourcePropertiesArray(value, property_type='position')
        
        
    @property
    def dimension(self):
        return self._dimension

    @dimension.setter
    def dimension(self, value):
        self._dimension = SourcePropertiesArray(value, property_type='dimension')
                 
    @property
    def axis(self):
        return self._axis
    
    @axis.setter
    def axis(self, value):
        self._axis = SourcePropertiesArray(value, property_type='axis')
        
    @property
    def angle(self):
        return self._angle
    
    @angle.setter
    def angle(self, value):
        self._angle = self._orientation.angle = value
        
        
    @property
    def orientation(self):
        return self._orientation

    @orientation.setter
    def orientation(self, value):
        if isinstance(value, SourceOrientation):
            self._orientation = value
        else:
            self._orientation = SourceOrientation(value[0], value[1])
    
    def show(self, savehtml=False, aswidget=False, cst=0.1):
        pmc = magpy.Collection(self.magnet)
        return displaySystemPlotly(pmc, savehtml=savehtml, aswidget=aswidget, cst=cst)
    
    def __repr__(self):
        return '''magnetization :  x: {}mT y: {}mT z: {}mT 
                 \ndimension :  x: {}mm y: {}mm z: {}mm 
                 \nposition :  x: {}mm y: {}mm z: {}mmm 
                 \naxis :  ({}, {}, {})
                 \nangle : {}'''.format(*self._magnetization, *self._dimension, *self._position, *self._axis, self._angle)
    
    def _ipython_display_(self):
        from IPython.display import HTML,display
        data = [['property', 'x' , 'y', 'z', 'unit'],
                ['magnetization'] + list(self._magnetization) + ['mT'],
                ['dimension'] + list(self._dimension)+ ['mm'],
                ['position'] + list(self._position) + ['mm'],
                ['orientation'] + list(self._axis) + [f'angle={self._angle:.2f}°'],
                ]

        display(HTML(
           '<table><tr>{}</tr></table>'.format(
               '</tr><tr>'.join(
                   '<td>{}</td>'.format('</td><td>'.join(str(col) for col in row)) for row in data)
               )
        ))
        

In [ ]:
class SphereMagnet:
    def __init__(self, mag=(0,0,1), dim=1, pos=(0,0,0), angle=0, axis=(0,0,1)):
        self._magnetization = SourcePropertiesArray(mag, property_type='magnetization')
        self._position = SourcePropertiesArray(pos, property_type='position')
        _test_float(dim)
        self._dimension = dim
        self._axis = SourcePropertiesArray(axis, property_type='axis')
        self._angle = angle
        self._orientation = SourceOrientation(angle, axis)
        self.name = 'Magnet'
        self.magnet = magpy.source.magnet.Sphere(mag, dim, pos, angle,  axis)
        
    @property
    def magnetization(self):
        return self._magnetization
    
    @magnetization.setter
    def magnetization(self, value):
        self._magnetization = SourcePropertiesArray(value, property_type='magnetization')
        
        
    @property
    def position(self):
        return self._position

    @position.setter
    def position(self, value):
        self._position = SourcePropertiesArray(value, property_type='position')
        
        
    @property
    def dimension(self):
        return self._dimension

    @dimension.setter
    def dimension(self, value):
        _test_float(value)
        self._dimension = value
            
            
    @property
    def axis(self):
        return self._axis
    
    @axis.setter
    def axis(self, value):
        self._axis = SourcePropertiesArray(value, property_type='axis')
        
    @property
    def angle(self):
        return self._angle
    
    @angle.setter
    def angle(self, value):
        self._angle = self._orientation.angle = value
        
        
    @property
    def orientation(self):
        return self._orientation

    @orientation.setter
    def orientation(self, value):
        if isinstance(value, SourceOrientation):
            self._orientation = value
        else:
            self._orientation = SourceOrientation(value[0], value[1])
    
    def show(self, cst=0.1):
        pmc = magpy.Collection(self.magnet)
        return pmc.displaySystemPlotly(cst=cst)
    
    def __repr__(self):
        return '''magnetization :  x: {}mT y: {}mT z: {}mT 
                 \ndimension :  x: {}mm y: {}mm z: {}mm 
                 \nposition :  x: {}mm y: {}mm z: {}mmm 
                 \naxis :  ({}, {}, {})
                 \nangle : {}'''.format(*self._magnetization, *self._dimension, *self._position, *self._axis, self._angle)
    
    def _ipython_display_(self):
        from IPython.display import HTML,display
        data = [['property', 'x' , 'y', 'z', 'unit'],
                ['magnetization'] + list(self._magnetization) + ['mT'],
                ['dimension'] + list(self._dimension)+ ['mm'],
                ['position'] + list(self._position) + ['mm'],
                ['orientation'] + list(self._axis) + [f'angle={self._angle:.2f}°'],
                ]

        display(HTML(
           '<table><tr>{}</tr></table>'.format(
               '</tr><tr>'.join(
                   '<td>{}</td>'.format('</td><td>'.join(str(col) for col in row)) for row in data)
               )
        ))
        

In [ ]:
m = BoxMagnet()
m2 = BoxMagnet()
m.angle = 8
m.orientation =  (1, (1,2,3))
m.position.x= 5
m

In [ ]:
MX = MY = 80  ;  MZ = 40 # [mm] magnet dimensions
ZB = 22 # [mm] vertical spacing between magnets
XB = 22/np.sqrt(3) # [mm] horizontal spacing between magnets

AX = (XB + MX)/2 # [mm] horizontal distance from the coordinate center to the magnet center
AZ = (ZB + MZ)/2 # [mm] vertical distance from the coordinate center to the magnet center

pmc = magpy.Collection()
mag1 = magpy.source.magnet.Box(mag=(0,0,-1),dim=(80,80,40), pos=(-AX,0,-AZ))
mag2 = magpy.source.magnet.Box(mag=(0,0,1),dim=(80,80,40), pos=(+AX,0,-AZ))
mag3 = magpy.source.magnet.Box(mag=(0,0,-1),dim=(80,80,40), pos=(-AX,0,+AZ))
mag4= magpy.source.magnet.Box(mag=(0,0,1),dim=(80,80,40), pos=(+AX,0,+AZ))
pmc = magpy.Collection(mag1,mag2,mag3,mag4)
mag4.setOrientation(45,(0,0,1))

In [ ]:
pmc.displaySystem()

In [ ]:
pmc.displaySystemPlotly(Nver=40, cst=0)

In [ ]:
collection_dict = dict(
    box = Box(mag=(0,0,-1),dim=(40,40,20), pos=(-40,0,0), angle=45, axis=(1,1,1)),
    cylinder = Cylinder(mag=(1,0,0),dim=(40,30), angle=10, pos=(40,0,40)),
    sphere = Sphere(mag=(0,1,0), dim=30, angle=20, axis=(1,0,0), pos=(40,0,-40)),
    line = Line(curr = 1, vertices = [(-30, 0.0, 30), (30, 0, 0)], pos=(10,-40,40), angle=-45),
    circular = Circular(curr=3, dim=50, angle=45, axis=(1,1,0), pos=(0,40,-50)),
    dipole = Dipole(moment=(1,2,-3), pos=(0,1,0)),
)

collection = magpy.Collection(*collection_dict.values())

In [ ]:
collection.displaySystemPlotly()